In [1]:
#  Airbnb London 2024 – Pre‑processing & Feature‑Engineering Pipeline
#  Author: Group‑3  (CSC‑40048 CW2)

import pandas as pd, numpy as np, re, os
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# 0 ── CONFIG 
CSV_PATH = r"C:\Visualization of Data Analytics\Airbnb_london.csv"

# 1 ── LOAD RAW DATA 
df = pd.read_csv(CSV_PATH)
print("Raw shape:", df.shape)            

# 2 ── COLUMN PRUNING  (drop heavy free‑text & web links) 
DROP_COLS = [
    "name","description","neighborhood_overview","picture_url",
    "host_url","scrape_id","thumbnail_url"
]
df = df.drop(columns=[c for c in DROP_COLS if c in df.columns], errors="ignore")

# 3 ── BATHROOMS: extract numeric from textual field 
if "bathrooms_text" in df.columns:
    df["bathrooms"] = (
        df["bathrooms_text"]
        .str.extract(r"(\d+\.?\d*)")
        .astype(float)
    )
    df.drop(columns="bathrooms_text", inplace=True, errors="ignore")

# 4 ── MISSING‑VALUE IMPUTATION 
review_cols = df.filter(like="review_scores").columns
df[review_cols] = df[review_cols].apply(lambda c: c.fillna(c.median()))

df["no_review_flag"] = df["last_review"].isna().astype(int)

# 5 ── DATE FEATURES 
df["last_review"] = pd.to_datetime(df["last_review"], errors="coerce")
df["days_since_review"] = (
    pd.Timestamp.today() - df["last_review"]
).dt.days.fillna(9999)

# 6 ── PRICE CLEAN & NUMERIC CAST 
df["price"] = (
    df["price"]
    .astype(str)
    .str.replace(r'[\$,]', '', regex=True)
    .astype(float)
)

# 7 ── OUTLIER CLIPPING (1st / 99th percentile) 
NUM_COLS = ["price","accommodates","review_scores_rating","bathrooms"]
existing_nums = [c for c in NUM_COLS if c in df.columns]

q_lo, q_hi = df[existing_nums].quantile(0.01), df[existing_nums].quantile(0.99)
df[existing_nums] = df[existing_nums].clip(q_lo, q_hi, axis=1)

# 8 ── FEATURE ENGINEERING 
df["price_ppp"] = df["price"] / df["accommodates"]

top10 = df["property_type"].value_counts().nlargest(10).index
df["property_major"] = np.where(
    df["property_type"].isin(top10), df["property_type"], "Other"
)

# 9 ── CATEGORICAL ONE‑HOT ENCODING 
CAT_COLS = ["room_type","neighbourhood_cleansed","property_major",
            "host_is_superhost"]
cats_present = [c for c in CAT_COLS if c in df.columns]
df = pd.get_dummies(df, columns=cats_present, drop_first=True)

# 10 ── SCALE CONTINUOUS FEATURES 
scale_cols = existing_nums + ["price_ppp","days_since_review"]
scaler = StandardScaler()
df[scale_cols] = scaler.fit_transform(df[scale_cols])

# 11 ── TRAIN‑TEST SPLIT 
y = df["price"]
X = df.drop(columns="price")
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.20, random_state=42
)

print("Train shape:", X_train.shape, "| Test shape:", X_test.shape)

# 12 ── SAVE CLEAN DATA (optional) 
out_dir = os.path.dirname(CSV_PATH)
df.to_csv(os.path.join(out_dir, "Airbnb_clean.csv"), index=False)
print("Clean CSV saved to:", os.path.join(out_dir, "Airbnb_clean.csv"))

Raw shape: (95144, 75)
Train shape: (76115, 113) | Test shape: (19029, 113)
Clean CSV saved to: C:\Visualization of Data Analytics\Airbnb_clean.csv
